In [164]:
import torch, model, importlib
importlib.reload(model)

context = [
    "Is there anyone who doesn't know Nancy?",
    "Do you - Do you know Nancy?",
    "Me?",
    "Mm-hmm",
    "I know Nancy"
]
speaker_change = [1, 0, 1, 0, 0] # 1 indicates a change in the speaker

In [165]:
# Preprocessing fait à la va-vite pour vérifier que l'encodeur ne renvoie pas d'erreur
vocab = [""] + list(set(" ".join(context).split()))
print(vocab)

['', "doesn't", 'I', 'Nancy', 'Do', 'anyone', 'there', 'Is', 'know', 'you', 'who', 'Nancy?', 'Me?', 'Mm-hmm', '-']


In [166]:
max_len = max([len(sentence.split()) for sentence in context])

X = torch.tensor([
    [vocab.index(word) for word in sentence.split()] + [0 for _ in range(max_len-len(sentence.split()))]
    for sentence in context
])
X

tensor([[ 7,  6,  5, 10,  1,  8, 11],
        [ 4,  9, 14,  4,  9,  8, 11],
        [12,  0,  0,  0,  0,  0,  0],
        [13,  0,  0,  0,  0,  0,  0],
        [ 2,  8,  3,  0,  0,  0,  0]])

In [167]:
X_ohe = torch.nn.functional.one_hot(X).unsqueeze(0).type(torch.float32)
X_ohe.shape
# (batch_size, sequence_length, max_sentence_length, input_size)

torch.Size([1, 5, 7, 15])

In [168]:
P = torch.tensor(speaker_change).unsqueeze(0)
P.shape
# (batch_size, sequence_length)

torch.Size([1, 5])

In [169]:
encoder = model.HierarchicalEncoder(input_size = len(vocab))
encoder(X_ohe, P).shape
# (bacth_size, 2*hidden_dim)

torch.Size([1, 256])

In [170]:
import torchinfo

torchinfo.summary(encoder)

Layer (type:depth-idx)                   Param #
HierarchicalEncoder                      --
├─GRU: 1-1                               111,360
├─DiscontinuedGRU: 1-2                   --
│    └─GRUCell: 2-1                      148,224
│    └─GRUCell: 2-2                      148,224
├─GRU: 1-3                               296,448
Total params: 704,256
Trainable params: 704,256
Non-trainable params: 0